# Segmenting and Clustering Neighborhoods in Toronto

## 1) Scrape Wikipedia

First I use the `pandas.read_html` method to parse html tables in the wikipedia page. 

In [1]:
import pandas as pd

In [31]:
toronto_DF = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
toronto_DF.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## 2) Cleaning Data

First we drop observations that have "Not assigned" in the Borough column.

In [32]:
# Dropping 'Not assigned' in Borough rows

filt = toronto_DF['Borough']!='Not assigned'
toronto_DF_filt = toronto_DF[filt]
print('Checking to see that there are no "Not assigned" groups')
toronto_DF_filt.groupby(['Borough']).count()

Checking to see that there are no "Not assigned" groups


,Postal Code,Neighbourhood
Borough,,
Central Toronto,9,9
Downtown Toronto,19,19
East Toronto,5,5
East York,5,5
Etobicoke,12,12
Mississauga,1,1
North York,24,24
Scarborough,17,17
West Toronto,6,6


We then check that there are no "Not assigned" values in the Neighbourhood column.

In [40]:
print('Number of Not assigned in column Neighbourhood:',sum(toronto_DF_filt['Neighbourhood']=='Not assigned'))

Number of Not assigned in column Neighbourhood: 0


We then check the shape of the Data Frame.

In [42]:
toronto_DF_filt.shape

(103, 3)